In [1]:
import pandas as pd
import nltk
import re
import string
from textblob import TextBlob
import numpy as np
import gensim
import os

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
import json
with open('intents.json') as file:
  dictionary = json.load(file)

[{'tag': 'greeting', 'patterns': ['Hello', 'Hi', 'Hey', 'Greetings'], 'responses': ['Hello! How can I assist you today?', 'Hi there! How may I help you?', 'Hey! Welcome. How can I be of service?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Goodbye', 'Bye', 'See you later', 'Farewell'], 'responses': ['Goodbye! If you have any more questions, feel free to ask.', 'Bye! Have a great day!', 'See you later! Safe travels.'], 'context_set': ''}, {'tag': 'thanks', 'patterns': ['Thank you', 'Thanks', 'Appreciate it'], 'responses': ["You're welcome! If you need anything else, feel free to ask.", "No problem! Let me know if there's anything else I can do for you.", 'Glad I could help! If you have more questions, just ask.'], 'context_set': ''}, {'tag': 'help', 'patterns': ['Help', 'Assistance', 'Can you help me?'], 'responses': ["Of course! I'm here to assist you. What do you need help with?", 'Certainly! What can I help you with today?', "Sure, I'm here to help. What do you require ass

In [24]:
data = pd.DataFrame(dictionary['intents'])
def toString(arr):
  info = ""
  for elements in arr:
    info += elements +" "
  return info

data['patterns']=data['patterns'].apply(toString)
print(data)

                   tag                                           patterns  \
0             greeting                            Hello Hi Hey Greetings    
1              goodbye                Goodbye Bye See you later Farewell    
2               thanks                    Thank you Thanks Appreciate it    
3                 help                  Help Assistance Can you help me?    
4           book_hotel  Book a hotel Reserve a room Hotel booking I wa...   
5   cancel_reservation  Cancel reservation I want to cancel my booking...   
6   check_availability  Check room availability Are there any rooms av...   
7   modify_reservation  Modify reservation Change booking details Upda...   
8     get_confirmation  Get booking confirmation Can I have my reserva...   
9         pricing_info  How much does a room cost? Tell me about your ...   
10           amenities  Tell me about hotel amenities What facilities ...   
11            feedback  Leave feedback Provide a review I want to shar...   

In [25]:
from nltk.corpus import stopwords

def remove_punc(text):
  exclude = set(string.punctuation)
  for char in exclude:
    text = text.replace(char,'')
  return text

def stop(text):
  new = []
  sw = stopwords.words('english')
  for words in text.split():
    if words in sw:
      new.append('')
    else:
      new.append(words)
  x = new[:]
  new.clear()
  return " ".join(x)


data['patterns']=data['patterns'].str.lower()
data['patterns']=data['patterns'].apply(remove_punc)
data['patterns']=data['patterns'].apply(stop)

print(data['patterns'])

0                                hello hi hey greetings
1                       goodbye bye see  later farewell
2                             thank  thanks appreciate 
3                               help assistance   help 
4     book  hotel reserve  room hotel booking  want ...
5     cancel reservation  want  cancel  booking canc...
6     check room availability    rooms available  ro...
7     modify reservation change booking details upda...
8     get booking confirmation     reservation detai...
9       much   room cost tell    prices    rate   night
10    tell   hotel amenities  facilities    list hot...
11    leave feedback provide  review  want  share  e...
Name: patterns, dtype: object


In [122]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
new = cv.fit_transform(data['patterns']).toarray()

In [128]:
from keras import Sequential
from keras.layers import Dense
output = [0,1,2,3,4,5,6,7,8,9,10,11]

model = Sequential()
model.add(Dense(8,input_shape=new[0].shape))
model.add(Dense(8))
model.add(Dense(8))
model.add(Dense(len(output),activation='softmax'))

In [129]:
model.summary()
model.compile(optimizer="Adam", loss = 'sparse_categorical_crossentropy',metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 888       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 12)                108       
                                                                 
Total params: 1140 (4.45 KB)
Trainable params: 1140 (4.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [130]:
model.fit(new,np.array(output),batch_size = 1,epochs =25)

Train on 12 samples
Epoch 1/25
12/12 [==============================] - 0s 2ms/sample - loss: 2.6115 - acc: 0.0000e+00
Epoch 2/25
12/12 [==============================] - 0s 1ms/sample - loss: 2.5212 - acc: 0.0000e+00
Epoch 3/25
12/12 [==============================] - 0s 2ms/sample - loss: 2.4337 - acc: 0.0000e+00
Epoch 4/25
12/12 [==============================] - 0s 1ms/sample - loss: 2.3536 - acc: 0.1667
Epoch 5/25
12/12 [==============================] - 0s 1ms/sample - loss: 2.2834 - acc: 0.3333
Epoch 6/25
12/12 [==============================] - 0s 1ms/sample - loss: 2.2069 - acc: 0.4167
Epoch 7/25
12/12 [==============================] - 0s 2ms/sample - loss: 2.1313 - acc: 0.4167
Epoch 8/25
12/12 [==============================] - 0s 2ms/sample - loss: 2.0561 - acc: 0.4167
Epoch 9/25
12/12 [==============================] - 0s 1ms/sample - loss: 1.9839 - acc: 0.4167
Epoch 10/25
12/12 [==============================] - 0s 1ms/sample - loss: 1.9118 - acc: 0.4167
Epoch 11/25
12/12

In [137]:
import random
message = "Can you update reservation"
test = cv.transform([message])
results = model.predict(test.toarray())
tag = data['tag'][np.argmax(results)]
resp = data.index[data['tag']==tag].tolist()[0]
print(random.choice(data['responses'][resp]))

No problem. Let me assist you in modifying your reservation. What changes do you have in mind?
